In [1]:
 ! pip install langchain
 ! pip install langchain_openai
 ! pip install langchain_community
 ! pip install pymongo
 ! pip install python-dotenv
 ! pip install azure-core
 ! pip install azure-cosmos

  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
     ---------------------------------------- 0.0/50.9 kB ? eta -:--:--
     ---------------------------------------- 50.9/50.9 kB 2.7 MB/s eta 0:00:00
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/867.6 kB ? eta -:--:--
   --------------------- ------------------ 460.8/867.6 kB 9.6 MB/s eta 0:00:01
   --------------------------------------- 867.6/867.6 kB 13.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ------------------------------ --------- 1.5/2.0 MB 49.2 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 31.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/302.9 kB ? eta -:--:--
   --------------------------------------- 302.9/302.9 kB 19.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.7 kB ? eta -:--:--
   --------------------------


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/41.9 kB ? eta -:--:--
     ---------------------------------------  41.0/41.9 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 41.9/41.9 kB 990.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/314.1 kB ? eta -:--:--
   ----------------------------- ---------- 235.5/314.1 kB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 314.1/314.1 kB 6.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/798.7 kB ? eta -:--:--
   --------------------------------------  788.5/798.7 kB 51.9 MB/s eta 0:00:01
   --------------------------------------- 798.7/798.7 kB 25.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/269.0 kB ? eta -:--:--
   --------------------------------------- 269.0/269.0 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.2.3
    Uninstalling openai-1.2.3:
      Successfully uninstalled openai-1.2.3



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import json
import openai
import pymongo

from dotenv import dotenv_values
from openai import AzureOpenAI

env_name = "example.env" 
config = dotenv_values(env_name)

openai.api_type = config['openai_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']

client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint = openai.api_base
)

In [3]:
# Connection string
mongo_conn = config['mongo_vcore_connection_string']
mongo_client = pymongo.MongoClient(mongo_conn)

# Database name
DATABASE_NAME = "ResturantChain"
db = mongo_client[DATABASE_NAME]

# Drop the database if it already exists (consider if this is really needed, as it will remove all existing data)
#mongo_client.drop_database(DATABASE_NAME)

# Collection names
collection_names = ["Contoso", "Contoso West", "Users"]

# Iterate through the collection names and create them if they do not exist
for collection_name in collection_names:
    if collection_name not in db.list_collection_names():
        # Creates a collection
        db.create_collection(collection_name)
        print(f"Created collection '{collection_name}'.")
    else:
        print(f"Using existing collection: '{collection_name}'.")

Created collection 'Contoso'.
Created collection 'Contoso West'.
Created collection 'Users'.
